In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('../src/')
from harp.bonsai import load_bonsai_config
load_bonsai_config(r"C:\git\AllenNeuralDynamics\aind-vr-foraging\Bonsai")
from harp.io import read_harp_bin, read_bytes_from_bin

In [2]:
import clr
clr.AddReference("Bonsai.Harp")
clr.AddReference("Harp.Behavior")
import Bonsai.Harp as harpcore
import Harp.Behavior as harpbehavior

In [3]:
def bytes_to_harp_message(bytes):
    return harpcore.HarpMessage(bytes)

def read_harp_bin_to_messages(filename: str) -> np.array:
    return array_to_harp_messages(read_bytes_from_bin(filename))

def array_to_harp_messages(data: np.array) -> np.array:
    vectorized_method = np.vectorize(bytes_to_harp_message)
    return np.apply_along_axis(lambda x: bytes_to_harp_message(x.tobytes()), axis=1, arr=data)

def message_to_payload_method(msg_or_address: harpcore.HarpMessage | int) -> any:
    if isinstance(msg_or_address, int):
        address = msg_or_address
    elif isinstance(msg_or_address, harpcore.HarpMessage):
        address = msg_or_address.Address
    else:
        raise ValueError("msg_or_address must be either a HarpMessage or an int")
    _type = harpbehavior.Device.RegisterMap[address]
    GetPayloadMethod = _type.GetMethod("GetTimestampedPayload")
    return lambda msg : GetPayloadMethod.Invoke(None, [msg])

def harp_messages_to_payloads(data: np.array, address = None) -> np.array:
    get_payload_method = message_to_payload_method(data[0].Address if address is None else address)
    vectorized_method = np.vectorize(get_payload_method)
    return vectorized_method(data)

def file_to_payloads(filename: str) -> np.array:
    data = read_harp_bin_to_messages(filename)
    return harp_messages_to_payloads(data)


In [4]:
root = r"Z:\scratch\vr-foraging\672107\20230920T125148"
device = "behavior"
register = 32
filepath = f"{root}//{device}//register__{register}.bin"

In [5]:

payloads = file_to_payloads(filepath)

print(payloads[0].Seconds)
print(payloads[0].Value)
print(payloads[0].Value.HasFlag(harpbehavior.DigitalInputs.DIPort0))


97727.047488
DIPort2
False


In [7]:

register = 44
filepath = f"{root}//{device}//register__{register}.bin"
payloads = file_to_payloads(filepath)

print(payloads[0].Seconds)
print(payloads[0].Value.Encoder)


Event 44 TimestampedS16@97725.06099 Length:3
Event 44 TimestampedS16@97725.06198 Length:3
Event 44 TimestampedS16@97725.06298 Length:3
Event 44 TimestampedS16@97725.064 Length:3
Event 44 TimestampedS16@97725.06499 Length:3
Event 44 TimestampedS16@97725.06598 Length:3
Event 44 TimestampedS16@97725.06698 Length:3
Event 44 TimestampedS16@97725.068 Length:3
Event 44 TimestampedS16@97725.06899 Length:3
Event 44 TimestampedS16@97725.06998 Length:3
Event 44 TimestampedS16@97725.07098 Length:3
Event 44 TimestampedS16@97725.072 Length:3
Event 44 TimestampedS16@97725.07299 Length:3
Event 44 TimestampedS16@97725.07398 Length:3
Event 44 TimestampedS16@97725.07498 Length:3
Event 44 TimestampedS16@97725.076 Length:3
Event 44 TimestampedS16@97725.07699 Length:3
Event 44 TimestampedS16@97725.07798 Length:3
Event 44 TimestampedS16@97725.07898 Length:3
Event 44 TimestampedS16@97725.08 Length:3
Event 44 TimestampedS16@97725.08099 Length:3
Event 44 TimestampedS16@97725.08198 Length:3
Event 44 TimestampedS

In [19]:
register = 44
filepath = f"{root}//{device}//register__{register}.bin"
read_harp_bin_to_messages(filepath)

array([<Bonsai.Harp.HarpMessage object at 0x000002470E5323C0>,
       <Bonsai.Harp.HarpMessage object at 0x000002470540A440>, ...,
      dtype=object)

In [20]:
read_harp_bin(filepath)

,0,1,2
Seconds,,,
97725.060992,2,-5,898
97725.061984,2,-5,896
97725.062976,4,-5,898
97725.064000,3,-5,896
97725.064992,1,-5,895
...,...,...,...
99417.280992,1,-2,902
99417.281984,4,-1,901
99417.282976,2,-2,901
